### creating Image representation

In [5]:
%pip install pandas numpy matplotlib

     ---------------------------------------- 11.1/11.1 MB 9.0 MB/s eta 0:00:00
     ---------------------------------------- 12.9/12.9 MB 9.2 MB/s eta 0:00:00
     ---------------------------------------- 8.1/8.1 MB 5.5 MB/s eta 0:00:00
     -------------------------------------- 509.2/509.2 KB 6.3 MB/s eta 0:00:00
  Using cached tzdata-2025.2-py2.py3-none-any.whl (347 kB)
     -------------------------------------- 111.1/111.1 KB 6.3 MB/s eta 0:00:00
  Using cached cycler-0.12.1-py3-none-any.whl (8.3 kB)
     ---------------------------------------- 2.7/2.7 MB 11.3 MB/s eta 0:00:00
     ------------------------------------- 221.2/221.2 KB 13.2 MB/s eta 0:00:00
  Using cached kiwisolver-1.4.8-cp310-cp310-win_amd64.whl (71 kB)
     ---------------------------------------- 2.2/2.2 MB 11.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Users\amirt\OneDrive\Desktop\RA work\Code\god\Scripts\python.exe -m pip install --upgrade pip' command.


In [6]:
# setting up the input path 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import random
from pathlib import Path

# Set up output directory
output_dir = Path("Data/New/AMP")
# Load the dataset
df = pd.read_csv("Data\Tuned\AMP.csv")


In [7]:
# Extract residue and interaction info
interaction_columns = df.columns[1:]
residue_info = [
    col.split('.') for col in interaction_columns
]  # e.g., ['ASP79', 'P', 'Cationic']
residues = [r[0] for r in residue_info]
interaction_types = [r[2] for r in residue_info]

In [12]:
#Randomly Select 200 Frames
selected_frames = df['Frame'].sample(n=50, random_state=42).values
selected_df = df[df['Frame'].isin(selected_frames)].sort_values("Frame").reset_index(drop=True)

In [13]:
# Define interaction and residue layout
unique_interactions = sorted(set(interaction_types))
interaction_to_row = {itype: idx for idx, itype in enumerate(unique_interactions)}
num_rows = len(unique_interactions)
num_cols = len(interaction_columns)

# For reverse lookup if needed later
residue_labels = [col.split('.')[0] for col in interaction_columns]
interaction_labels = [col.split('.')[-1] for col in interaction_columns]

def save_heatmap_and_npy(row, frame_index, save_dir):
    # Create binary matrix (interaction_type x residue)
    binary_array = np.zeros((num_rows, num_cols), dtype=np.uint8)
    for col_idx, col in enumerate(interaction_columns):
        interaction = col.split('.')[-1]
        row_idx = interaction_to_row[interaction]
        binary_array[row_idx, col_idx] = row[col]

    # Save heatmap-style image (no labels)
    plt.figure(figsize=(8, 6))
    plt.imshow(binary_array, cmap='plasma', aspect='auto')
    plt.axis('off')
    plt.title(f"AMP_frame_{frame_index}.npy")
    plt.tight_layout()
    plt.savefig(save_dir / f"AMP_frame_{frame_index}.png", dpi=150)
    plt.close()

    # Save metadata-enhanced npy file
    output = {
        "binary_array": binary_array,
        "residue_labels": residue_labels,
        "interaction_labels": unique_interactions
    }
    np.save(save_dir / f"AMP_frame_{frame_index}.npy", output)

# Run for all 200 selected frames
for idx, row in selected_df.iterrows():
    save_heatmap_and_npy(row, idx, output_dir)

print(f"✅ Saved 200 heatmaps and metadata-enhanced .npy files in: {output_dir}")


✅ Saved 200 heatmaps and metadata-enhanced .npy files in: Data\New\AMP
